

---
**I have used Eurparl monolingual dataset for english language with 9672 text files.**

*1. The whole dataset is stored as one sentence per row and BertLM model is fine tuned on same and saved in the directory for 300000 steps*


---



Downloading important packages  **[Run these cells]**
---



In [1]:
! pip install beautifulsoup4  ## beautifulsoup package to handle the tags in the given text files

In [2]:
pip install nltk  ## downloading nltk package for stopwords and stemming 

In [3]:
pip install transformers ## downloading transformers package to fine tune the pretrained Bert model on Language modeling task

     |████████████████████████████████| 1.8MB 15.7MB/s 
     |████████████████████████████████| 890kB 55.9MB/s 
     |████████████████████████████████| 2.9MB 47.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5865c014a00a1403d9f5084c94938234a2cc0119ee133e912ff082404523f1cc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Importing libraries **[Run these cells]**
---



In [ ]:
import torch 
import pickle
import os
import re
import tqdm as tqdm
import pandas as pd
import numpy as np
import random
import nltk
from google.colab import drive
from pathlib import Path
from bs4 import BeautifulSoup
# from functools import reduce
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

drive.mount('/content/drive')

In [6]:
root ='/content/drive/My Drive/Colab Notebooks/Task2-Fine Tune BERT/'

Preprocessing and storing data in pickle 
---



In [ ]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/Task2-Fine Tune BERT'

In [ ]:
############################  pre-processing the texts from each file and storing in dataframe with one sentence as one row  ######
#  Preprocessing steps:
#  1. Remove chapter names from the text
#  2. Split text on newline --> generates list of all rows
#  3. Remove Blank strings from output of previous step --> Contains non-empty rows
#  4. Splitting the senteces based on fullstop followed by a space --> Splitting sentences present in each row
#  5. Flatten the nested list into one list
#  6. Store the list in dataframe with one sentence per row

df = pd.DataFrame(columns=['sentences'])
count = 0
os.chdir(os.path.join(os.getcwd(),'en/'))
files = os.listdir()
for item in tqdm.tqdm(files):
  # print(item)
  contents = Path(os.path.join(os.getcwd(),item)).read_text()
  contents = re.sub(r'(\<CHAPTER.*?)\n(.*)(\n<SPEAKER.*)','\\1\\3',contents) ##remove the chapter names
  soup=BeautifulSoup(contents)
  text = str(soup.text).split('\n') # splits the text on new line
  text = list(filter(None, text)) # removes blank strings
  text = [sentence.split('. ') for sentence in text] # split sentences on fullstop followed by space

  text = reduce(lambda x,y: x+y,text) #convert nested lists into one list
  df2 = pd.DataFrame(text,columns=['sentences'])
  df = df.append(df2)


100%|██████████| 9672/9672 [3:07:24<00:00,  1.16s/it]


In [ ]:
# ############################### save the dataframe as pickle file ########################################
open_file = open(os.path.join(root,'dataset_in_dataframe.pkl'), "wb")
pickle.dump(df, open_file)

# BERT fine tuning 
---



Reading the stored dataset in form of sentences per row (dataframe) and Determining the common sentence length
---



In [ ]:
############################## read the saved pickle file and store the dataset in variable df #########################################

open_file = open(os.path.join(root,'dataset_in_dataframe.pkl'), "rb")
df = pickle.load(open_file)

In [ ]:
############################## reindex the dataframe index to length of dataframe
index = [i for i in range(0,len(df))]
np.max(index)
df = df.reset_index(drop=True)
df = df.reindex(index)

In [ ]:
length = [len(sentence.split()) for i,sentence in enumerate(df['sentences'])]
max(set(length), key = length.count)
# np.bincount(np.array(length)).argmax()


17

In [ ]:
max_seq_length = 30 #512 

Importing nesessary methods for Bert training and Configuration
---



In [ ]:
########### import important methods related to Bert model ########################

from transformers import (
    BertTokenizer,    # for tokenizing the text
    BertConfig,       # for retrieving the model configuration
    BertForMaskedLM,  # Masked language model for letting the model understand our corpus [fine-tuning]
    DataCollatorForLanguageModeling,  # for language modeling task. Takes list of samples from a Dataset and collate them into a batch
    Trainer,          # Used to train the specified model on given dataset and customized training arguments
    TrainingArguments # Used to customize the training parameters like epochs, save logs etc.
)

In [ ]:
######### Getting configuration of pretrained model 'bert-base-uncased' 
######### [uncased: no difference between english and English]

config = BertConfig.from_pretrained('bert-base-uncased')

In [ ]:
######### Getting tokenizer from pretrained model 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
######### getting pretrained masked language model 'bert-base-uncased'

model = BertForMaskedLM.from_pretrained('bert-base-uncased',
                                        config= config,
                                        from_tf=bool(".ckpt" in 'bert-base-uncased')
                                        )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
# tokenizer.model_max_length

In [ ]:
###### Instantiating Data collator 
# Used to form a batch from a list of elements of train_dataset or eval_dataset
# Provided the pretrained bert tokenizer and Masking 15% of the tokens of a sentence

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=float(0.15))

Text Tokenization and setting train and validation sets
---



In [ ]:
############## Tokenization function to tokenize the examples
# Tokenizer used is bert tokenizer and customized according to needs

def tokenize_function(examples):
  return tokenizer(examples,   # each sentence 
                   padding= 'max_length',   # padding the sentences upto max length. other options [True(pad upto max lenth) ; False(no padding)]
                   truncation=True,         # truncating long sentences with more than 30 tokens
                   max_length=max_seq_length, # maximum legth set for each sentence is upto 30 tokens
                   return_special_tokens_mask=True)  # return special tokens mask information.

In [ ]:
# Tokenizing the whole dataset 
tokenized_datasets_whole =  df['sentences'].map(tokenize_function)   # method to tokenise the trainset

In [ ]:
#### Storing tokenized dataset as pickle 
open_file = open(os.path.join(root,'tokenized_whole_dataset.pkl'), "wb")
pickle.dump(tokenized_datasets_train, open_file)


In [ ]:
#### Reading tokenized dataset from pickle

open_file = open(os.path.join(root,'tokenized_whole_dataset.pkl'), "rb")
tokenized_datasets_whole= pickle.load(open_file)

Dividing the datset into 8:2 ratio 

In [ ]:
##################### 80% data taken as train set and 20% as validation set

size_of_train_set =int(0.8 * len(tokenized_datasets_whole)) 
print(len(df),size_of_train_set)

2266612 1813289


In [ ]:
tokenized_datasets_train = tokenized_datasets_whole[:size_of_train_set].reset_index(drop=True)

In [ ]:
tokenized_datasets_val = tokenized_datasets_whole[size_of_train_set:].reset_index(drop=True)

Fine Tuning Bert on Europarl dataset
---



In [ ]:
## Changing directory to folder where you would like to store trained model [final and model at checkpoints]
os.chdir('/content/drive/My Drive/Colab Notebooks/Task2-Fine Tune BERT/fine-tuned-model')
stored_model_path = os.getcwd()
stored_model_path

'/content/drive/My Drive/Colab Notebooks/Task2-Fine Tune BERT/fine-tuned-model'

In [ ]:
### Instantiating trainer with following parameters:
# pretrained 'bert-base-uncased' model
# tokenized train set
# tokenized validation set
# pretrained 'bert-base-uncased' tokenizer
# previously initialised data_collator
# Training Arguments: output directory to save model at checkpoints; steps at which the trained model will be saved and related parameters wil be logged

trainer = Trainer(
        model=model,
        # args=training_args,
        train_dataset=tokenized_datasets_train, #tokenized_datasets["train"] if training_args.do_train else None,
        eval_dataset=tokenized_datasets_val, #tokenized_datasets["validation"] if training_args.do_eval else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        args= TrainingArguments(output_dir = os.getcwd(),logging_steps=300000,save_steps=300000)
    )

In [ ]:
#### Fine tune 'bert-base-uncased' model with train function
train_result = trainer.train('bert-base-uncased')

Step,Training Loss


**Saving the model :** *I couldnt train on whole train dataset due to limited resources and hence used the last logged trained model[model logged at 300000 steps] for predicting the word to replace the desired word*

In [ ]:
trainer.save_model(output_dir=path)   # you can use this method to store the trained model at desired location by giving the path

# Load the saved Model and replace the desired word **[Run these cells]**

In [7]:
from transformers import (
    BertTokenizer,    # for tokenizing the text
    BertConfig,       # for retrieving the model configuration
    BertForMaskedLM,  # Masked language model for letting the model understand our corpus [fine-tuning]
    DataCollatorForLanguageModeling,  # for language modeling task. Takes list of samples from a Dataset and collate them into a batch
    Trainer,          # Used to train the specified model on given dataset and customized training arguments
    TrainingArguments # Used to customize the training parameters like epochs, save logs etc.
)

In [8]:
def Fine_tuned_parameters(root_dir):
  stored_model_path = root_dir+'fine-tuned-model/checkpoint-300000/'
  fine_tuned_config = BertConfig.from_pretrained(stored_model_path)
  fine_tuned_tokenizer = BertTokenizer.from_pretrained(stored_model_path)
  fine_tuned_model = BertForMaskedLM.from_pretrained(stored_model_path,
                                        config= fine_tuned_config
                                        #from_tf=bool(".ckpt" in 'bert-base-uncased')
                                        )
  return fine_tuned_config,fine_tuned_tokenizer,fine_tuned_model

In [9]:
## Tokenization function with fine tuned tokenizer

def fine_tuned_tokenize_function(examples,fine_tuned_tokenizer):
  # print (examples)
  # tokens = tokenizer([sentence for sentence in data_files['train'][:10]], return_special_tokens_mask=True)
  return fine_tuned_tokenizer(examples,
                   padding= 'max_length',
                   truncation=True,
                   max_length=30,
                   return_special_tokens_mask=True)

In [10]:
def Output(sentence, word,root_dir):
  fine_tuned_config,fine_tuned_tokenizer,fine_tuned_model = Fine_tuned_parameters(root_dir)
  # tokenized_sentence = fine_tuned_tokenizer(sentence,
  #                                           padding= 'max_length',
  #                                           truncation=True,
  #                                           max_length=30,
  #                                           return_special_tokens_mask=True)

  tokenized_sentence = fine_tuned_tokenize_function(sentence,fine_tuned_tokenizer)
  # tokenized_sentence =  (pd.core.series.Series(sentence),fine_tuned_tokenizer).map(fine_tuned_tokenize_function)
  fine_tuned_model.eval()

  outputs = fine_tuned_model(torch.tensor([tokenized_sentence['input_ids']]),token_type_ids = torch.tensor([tokenized_sentence['token_type_ids']]))

  #####determine position of word to be replaced 
  position = [word for word in sentence.split()].index(word)

  ##### determine the possible alternatives of the word to be replaced
  predictions = outputs[0][0][position+1]
  predicted_ids = torch.argsort(predictions, descending=True)[:20]
  predicted_tokens = fine_tuned_tokenizer.convert_ids_to_tokens(list(predicted_ids))

  # print('predicted tokens :',predicted_tokens)

  ##### replacing the word with new word with similar meaning 
  # stemming the predicted and given words to their root form
  lancaster=LancasterStemmer()
  stemmed_predicted_tokens = [ lancaster.stem(i) for i in predicted_tokens]
  stemmed_word = lancaster.stem(word)

  # replacing the word 
  replaced_sentence = sentence
  for ind, token in enumerate(stemmed_predicted_tokens):
    if token not in stopwords.words('english') and not token == stemmed_word and not '#' in token and token not in stemmed_word.lower() and stemmed_word.lower() not in token:
          replaced_sentence = replaced_sentence.replace(sentence.split()[position],predicted_tokens[ind])
          break

  return replaced_sentence

In [21]:
os.chdir('/content/drive/My Drive/Colab Notebooks/Internship submission/Task2-Fine Tune BERT/fine-tuned-model/checkpoint-300000/')#Task2-Fine Tune BERT/fine-tuned-model/checkpoint-300000/')
os.listdir()#fine_tuned_config = BertConfig.from_pretrained(stored_model_path)

['checkpoint-300000', 'runs']

# **Enter the sentence and the word you want to change**

In [22]:
#@title **Please run the cell to input the sentence and the word to be replaced** { display-mode: "form" }
root_dir = input('1. Please enter the root directory where the folder with files is stored \n[ex: /content/drive/My Drive/Colab Notebooks/Task2-Fine Tune BERT/]  :  ')

input_sentence = input("\n2. Please enter the input sentence  : ")
input_word = input('\n3. Please enter the word to be changed : ')

print('\n\n####### Fetching new sentence ;) ########\n')
replaced_sentence = Output(input_sentence,input_word,root_dir)

print('\n4. Original sentence : ',input_sentence)
print('\n5. New Sentence with replaced word : ' , replaced_sentence)



1. Please enter the root directory where the folder with files is stored 
[ex: /content/drive/My Drive/Colab Notebooks/Task2-Fine Tune BERT/]  :  /content/drive/My Drive/Colab Notebooks/Internship submission/Task2-Fine Tune BERT/

2. Please enter the input sentence  : This sort of service and difficulty in getting to Strasbourg on a Monday is quite hopeless

3. Please enter the word to be changed : difficulty


####### Fetching new sentence ;) ########


4. Original sentence :  This sort of service and difficulty in getting to Strasbourg on a Monday is quite hopeless

5. New Sentence with replaced word :  This sort of service and delay in getting to Strasbourg on a Monday is quite hopeless


**`sample:output `**

1. Please enter the root directory where the folder with files is stored 
[ex: /content/drive/My Drive/Colab Notebooks/Task2-Fine Tune BERT/]  :  /content/drive/My Drive/Colab Notebooks/Task2-Fine Tune BERT/

2. Please enter the input sentence  : This sort of service and difficulty in getting to Strasbourg on a Monday is quite hopeless

3. Please enter the word to be changed : difficulty


####### Fetching new sentence ;) ########


4. Original sentence :  This sort of service and difficulty in getting to Strasbourg on a Monday is quite hopeless

5. New Sentence with replaced word :  This sort of service and delay in getting to Strasbourg on a Monday is quite hopeless